In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditriskmodeling-preprocessed/PreprocessedData.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("/kaggle/input/creditriskmodeling-preprocessed/PreprocessedData.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 42064 non-null  int64  
 1   pct_tl_open_L6M            42064 non-null  float64
 2   pct_tl_closed_L6M          42064 non-null  float64
 3   Tot_TL_closed_L12M         42064 non-null  int64  
 4   pct_tl_closed_L12M         42064 non-null  float64
 5   Tot_Missed_Pmnt            42064 non-null  int64  
 6   CC_TL                      42064 non-null  int64  
 7   Home_TL                    42064 non-null  int64  
 8   PL_TL                      42064 non-null  int64  
 9   Secured_TL                 42064 non-null  int64  
 10  Unsecured_TL               42064 non-null  int64  
 11  Other_TL                   42064 non-null  int64  
 12  Age_Oldest_TL              42064 non-null  int64  
 13  Age_Newest_TL              42064 non-null  int

In [4]:
df.head()

,Unnamed: 0,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,...,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
0,0,0.000,0.0,0,0.000,0,0,0,4,1,...,0.000,0.0,1,0,Married,2,M,PL,PL,P2
1,1,0.000,0.0,0,0.000,0,0,0,0,0,...,0.000,0.0,0,0,Single,3,F,ConsumerLoan,ConsumerLoan,P2
2,2,0.125,0.0,0,0.000,1,0,0,0,2,...,0.000,0.0,1,0,Married,1,M,ConsumerLoan,others,P2
3,3,0.000,0.0,0,0.000,0,0,0,0,3,...,0.000,0.0,0,0,Married,4,M,AL,AL,P1
4,4,0.000,0.0,1,0.167,0,0,0,0,6,...,0.429,0.0,1,0,Married,2,M,ConsumerLoan,PL,P3


In [5]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])
print(df_encoded.shape)
df_encoded.info()

(42064, 56)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 56 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    42064 non-null  int64  
 1   pct_tl_open_L6M               42064 non-null  float64
 2   pct_tl_closed_L6M             42064 non-null  float64
 3   Tot_TL_closed_L12M            42064 non-null  int64  
 4   pct_tl_closed_L12M            42064 non-null  float64
 5   Tot_Missed_Pmnt               42064 non-null  int64  
 6   CC_TL                         42064 non-null  int64  
 7   Home_TL                       42064 non-null  int64  
 8   PL_TL                         42064 non-null  int64  
 9   Secured_TL                    42064 non-null  int64  
 10  Unsecured_TL                  42064 non-null  int64  
 11  Other_TL                      42064 non-null  int64  
 12  Age_Oldest_TL                 42064 non-null  in

In [6]:
df_encoded.describe()

,Unnamed: 0,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,...,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION
count,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,...,42064.000000,4.206400e+04,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000
mean,21031.500000,0.179032,0.097783,0.825504,0.160365,0.525746,0.145921,0.076241,0.328000,2.921334,...,1.230458,2.692990e+04,110.345783,0.102962,0.193063,0.195497,0.064186,0.252235,0.056580,2.313689
std,12142.975198,0.278043,0.210957,1.537208,0.258831,1.106442,0.549314,0.358582,0.916368,6.379764,...,2.069461,2.084300e+04,75.629967,0.303913,0.394707,0.367414,0.225989,0.434300,0.231042,0.871070
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,10515.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.800000e+04,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,21031.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,2.400000e+04,92.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,31547.250000,0.333000,0.100000,1.000000,0.250000,1.000000,0.000000,0.000000,0.000000,3.000000,...,2.000000,3.100000e+04,131.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000
max,42063.000000,1.000000,1.000000,33.000000,1.000000,34.000000,27.000000,10.000000,29.000000,235.000000,...,42.000000,2.500000e+06,1020.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000


In [7]:
print(df_encoded.max())

Unnamed: 0                        42063
pct_tl_open_L6M                     1.0
pct_tl_closed_L6M                   1.0
Tot_TL_closed_L12M                   33
pct_tl_closed_L12M                  1.0
Tot_Missed_Pmnt                      34
CC_TL                                27
Home_TL                              10
PL_TL                                29
Secured_TL                          235
Unsecured_TL                         55
Other_TL                             80
Age_Oldest_TL                       385
Age_Newest_TL                       359
time_since_recent_payment          6065
max_recent_level_of_deliq           900
num_deliq_6_12mts                    20
num_times_60p_dpd                    52
num_std_12mts                       122
num_sub                              41
num_sub_6mts                          5
num_sub_12mts                        12
num_dbt                              35
num_dbt_12mts                        12
num_lss                              72


In [8]:
# Data processing
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [9]:
# MinMax Scaler for data scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# label encoding for target feature
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Hyperparameter tuning in xgboost using grid search cv

In [10]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [11]:
from xgboost import XGBClassifier

# Define the XGBClassifier with the initial set of hyperparameters
xgb_model = XGBClassifier(objective='multi:softmax', num_class=4, booster='gbtree', n_jobs  = -1)

In [12]:
# Define the parameter grid for hyperparameter tuning

param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.5, 0.8, 1.0],
    'reg_lambda': [1, 4, 7, 10],
    'reg_alpha': [1, 4, 7, 10]
}

In [13]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster='gbtree',
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=N...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=-1, num_class=4,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                         'max_depth': [3, 5, 7, 10],
                         'n_estimators': [50, 100, 150, 200],
                         'reg_alpha': [1, 4, 7, 10],
                         'reg_lambda': [1, 4, 7, 10],
                         'subsample': [0.5, 0.8, 1.0]},
             scoring='accuracy')

In [14]:
# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Test Accuracy:", accuracy)

Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 150, 'reg_alpha': 1, 'reg_lambda': 10, 'subsample': 0.5}
Test Accuracy: 0.7802210864138833


In [15]:
accuracy = best_model.score(x_train, y_train)
print("Train Accuracy:", accuracy)

Train Accuracy: 0.800332828147752
